In [ ]:
import os, urllib.request, json, shapefile, shutil, subprocess, csv, fiona, shapely, geopandas, pandas
import psycopg2.extras as pgextras
from osgeo import gdal, ogr, osr
from datetime import datetime

In [ ]:
os.environ['SHAPE_ENCODING'] = "utf-8"
basePath = 'C:\\saprog\\projeto'
outputPath = 'C:\\saprog\\projeto\\output'
os.chdir(basePath)

def deleteAndCreateOutput():
    print ('A limpar e criar directoria de outputs...')
    outDir = os.path.join(basePath, 'output')
    if os.path.isdir(outDir):
        shutil.rmtree(outDir)
    os.mkdir(outDir)

deleteAndCreateOutput()

In [ ]:
print('A ler shapefile...')
caop = geopandas.read_file('caop.shp',encoding='utf-8')
# caop.plot()
# caop.head()
# gdf = caop.loc[caop['Concelho'] == 'Bombarral']
# gdf.head()

In [ ]:
print('A executar Dissolve...')
distritos = caop.dissolve(by = 'Distrito')
# distritos.plot()
# distritos.head(1)

In [ ]:
print('A extrair centroides das regiões admninistrativas...')
shpOutput = os.path.join(outputPath, 'centroides.shp')

# dataProj = distritos.to_crs(epsg = 3763)
# centroides = dataProj.centroid
centroides = distritos.centroid
centroides.to_file(shpOutput, encoding='utf-8')
# centroides.plot()
# centroides.head()

In [ ]:
coordx = centroides.x.to_dict()
coordy = centroides.y.to_dict()
# print(coordx)
# print(coordy)

def getCoordinates(dicX, dicY):
    coord = [dicX, dicY]
    coordDic = {}
    for i in dicX.keys():
        coordDic[i] = tuple(coordDic[i] for coordDic in coord)
    return coordDic

print('A extrair coodenadas geográficas...')
coord = getCoordinates(coordx, coordy)

In [ ]:
# Carregamento de shapefile na bd
# command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
#           "-f", "PostgreSQL",
#           "PG:host=localhost user=postgres dbname=meteo password=3763",
#           shpOutput,
#           "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
#           " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", "centroides", "-overwrite"]
# subprocess.check_call(command)

In [41]:
apiKey = '44cfad7f82ec61d3f420de3201b703d4'

def getWeather(coordDic, apikey):
    forecast = []
    for item in coord.items():
        lat = str(item[1][1])
        long = str(item[1][0])
        url = 'https://api.openweathermap.org/data/2.5/onecall?lat=' + lat + '&lon=' + long + \
            '&exclude=minutely,hourly,daily,alerts&appid=' + apiKey + '&units=metric'
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
            districtForecast = {}
            districtForecast['distrito'] = item[0]
            districtForecast['forecast_date'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%d-%m-%Y')
            districtForecast['forecast_time'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%H:%M:%S')
            main = data.get('current').get('weather')
            for item in main:
                districtForecast['weather_desc'] = item.get('main')
            districtForecast['temperature'] = data.get('current').get('temp')
            districtForecast['feels_like'] = data.get('current').get('feels_like')
            districtForecast['pressure'] = data.get('current').get('pressure')
            districtForecast['humidity'] = data.get('current').get('humidity')
            districtForecast['dew_point'] = data.get('current').get('dew_point')
            districtForecast['ultrav_index'] = data.get('current').get('uvi')
            districtForecast['wind_speed'] = data.get('current').get('wind_speed')
            districtForecast['wind_deg'] = data.get('current').get('wind_deg')
            forecast.append(districtForecast)
    return forecast

forecast_df = pandas.DataFrame(getWeather(coord, apiKey))
forecast_df.head(-1)

,distrito,forecast_date,forecast_time,weather_desc,temperature,feels_like,pressure,humidity,dew_point,ultrav_index,wind_speed,wind_deg
0,Aveiro,14-03-2021,17:38:12,Clear,14.70,8.91,1026,72,9.71,0.00,8.23,350
1,Beja,14-03-2021,17:38:13,Clear,19.00,15.15,1024,42,5.81,0.00,4.12,20
2,Braga,14-03-2021,17:38:13,Clouds,15.56,13.08,1026,53,6.03,0.00,2.23,351
3,Bragança,14-03-2021,17:38:13,Clouds,11.91,10.22,1025,65,5.55,0.00,0.96,353
4,Castelo Branco,14-03-2021,17:38:13,Clear,16.67,13.23,1025,44,4.37,0.00,3.13,76
5,Coimbra,14-03-2021,17:38:14,Clear,15.25,9.31,1026,67,9.17,0.00,8.23,360
6,Faro,14-03-2021,17:38:14,Clear,18.00,15.85,1022,59,9.88,0.10,3.09,250
7,Guarda,14-03-2021,17:38:14,Clear,16.67,13.23,1025,44,4.37,0.00,3.13,76
8,Leiria,14-03-2021,17:38:14,Clear,14.45,8.32,1026,67,8.41,0.10,8.23,360
9,Lisboa,14-03-2021,17:38:15,Clear,15.48,10.73,1023,41,2.30,0.11,4.47,16


In [42]:
password = open(basePath+'\\pw.txt', 'r').readline()
# password = str(password)

con = psycopg2.connect(dbname='meteo', user='postgres', password=password, host='localhost', port='5432')

def dfToPgSQL(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        pgextras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

execute_values(con, forecast_df, 'forecast')

execute_values() done
